# Working with Galaxy objects

Synthesizer contains two types of galaxy:

- A particle `Galaxy` for working with galaxies comprised of individual stars, gas and/or black holes (`synthesizer.particle.galaxy.Galaxy`).
- A parametric `Galaxy` for working with parametric models described by SFZH and (when necessary) morphologies (`synthesizer.parametric.galaxy.Galaxy`).

As such instantiating and working with `Galaxy`s has the potential to lead to confusion. To simplify things for the user we provide a galaxy "factory function" which will always return the correct galaxy based on the arguments passed by the user while raising errors or warnings for incompatible combinations.

Below we set up some of the necessary stuff we will need to demonstrate the use `Galaxy`s and the factory function. A description of the significance can be found elsewhere in the documentation.

In [1]:
import numpy as np
from unyt import Myr

from synthesizer.grid import Grid
from synthesizer.parametric.sfzh import SFH, ZH, generate_sfzh
from synthesizer.particle.stars import sample_sfhz
from synthesizer.particle import Stars
from synthesizer.particle.particles import CoordinateGenerator

# Define the grid
grid_name = "test_grid"
grid_dir = "../../../tests/test_grid/"
grid = Grid(grid_name, grid_dir=grid_dir)

# Define the metallicity history
Z_p = {"Z": 0.01}
Zh = ZH.deltaConstant(Z_p)

# Define the star formation history
sfh_p = {"duration": 100 * Myr}
sfh = SFH.Constant(sfh_p)

# Initialise the SFZH object
sfzh = generate_sfzh(grid.log10age, grid.metallicity, sfh, Zh, stellar_mass=10**9)

# Define the number of stellar particles we want
n = 10000

# Sample the SFZH, producing a Stars object
# we will also pass some keyword arguments for some example attributes
stars = sample_sfhz(
    sfzh, 
    n, 
    current_masses=np.full(n, 10**8.7 / n), 
    redshift=1, 
    initial_mass=np.full(n, 10**6)
)

## Creating a Galaxy

Now that we have the building blocks of both a particle and parametric galaxy we can import the factory function and get our galaxies. 

To do so we simply pass the factory function the arguments for the desired type of `Galaxy`. In the case of a particle based galaxy these are objects from the `particle` module including `Stars`, `Gas`, and `BlackHoles`. For a parametric galaxy these are `BinnedSFZH` and (when applicable) morphology objects such as `Sersic2D`.

Note that a particle `Galaxy` can be intialised with any combination of `Stars`, `Gas`, or `BlackHoles` no single particle object is required in all case. Each is a key word argument which defaults to `None` Conversly, a parametric `Galaxy` must

In [2]:
from synthesizer import galaxy

# Get a particle galaxy
part_gal = galaxy(stars=stars, gas=None, black_holes=None, redshift=1)
print(type(part_gal))

# Get a parametric galaxy
param_gal = galaxy(stars=sfzh, morph=None, redshift=1)
print(type(param_gal))

<class 'synthesizer.particle.galaxy.Galaxy'>
<class 'synthesizer.parametric.galaxy.Galaxy'>


Of course you are free to avoid this abstraction and explictly instantiate the desired `Galaxy`.

In [3]:
from synthesizer.particle import Galaxy

# Get a particle galaxy
part_gal = Galaxy(stars=stars, gas=None, black_holes=None, redshift=1)

Or for a parametric galaxy.

In [4]:
from synthesizer.parametric import Galaxy

# Get a parametric galaxy
param_gal = galaxy(stars=sfzh, morph=None, redshift=1)

Once a galaxy has been created there are numerous methods for creating and plotting different types of spectra (e.g. [CAMELS example](../cosmo/galaxy_obj.ipynb)), making images or maps (e.g. [imaging docs](../imaging/particle_imaging.ipynb)), or computing properties. 